Source: https://www.kaggle.com/code/jhoward/why-you-should-use-a-framework

In [ ]:
!pip install kaggle

In [ ]:
import os
from pathlib import Path

competition = "titanic"  # Change this to any Kaggle competition name
iskaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")

if iskaggle:
    path = Path(f"../input/{competition}")
else:
    import kaggle

    # Save data in `data/` directories, that are gitignored
    path = Path("data")

    if not path.exists():
        import zipfile

        kaggle.api.competition_download_cli(competition=competition, path=str(path))
        zipfile.ZipFile(f"{path}/{competition}.zip").extractall(path)

print(f"Ready for competition: {competition}")

In [ ]:
import pandas as pd

In [ ]:
from fastai.tabular.all import *

pd.options.display.float_format = "{:.2f}".format
set_seed(42)

In [ ]:
df = pd.read_csv(path / "train.csv")

In [ ]:
def add_features(df):
    df["LogFare"] = np.log1p(df["Fare"])
    df["Deck"] = df.Cabin.str[0].map(
        dict(A="ABC", B="ABC", C="ABC", D="DE", E="DE", F="FG", G="FG")
    )
    df["Family"] = df.SibSp + df.Parch
    df["Alone"] = df.Family == 0
    df["TicketFreq"] = df.groupby("Ticket")["Ticket"].transform("count")
    df["Title"] = df.Name.str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
    df["Title"] = df.Title.map(dict(Mr="Mr", Miss="Miss", Mrs="Mrs", Master="Master"))


add_features(df)

In [ ]:
splits = RandomSplitter(seed=42)(df)

In [ ]:
dls = TabularPandas(
    df,
    splits=splits,
    procs=[Categorify, FillMissing, Normalize],
    cat_names=["Sex", "Pclass", "Embarked", "Deck", "Title"],
    cont_names=["Age", "SibSp", "Parch", "LogFare", "Alone", "TicketFreq", "Family"],
    y_names="Survived",
    y_block=CategoryBlock(),
).dataloaders(path=".")

In [ ]:
learn = tabular_learner(dls, metrics=accuracy, layers=[10, 10])

In [ ]:
learn.lr_find(suggest_funcs=(slide, valley))

In [ ]:
learn.fit(16, lr=0.03)

In [ ]:
tst_df = pd.read_csv(path / "test.csv")
tst_df["Fare"] = tst_df.Fare.fillna(0)
add_features(tst_df)

In [ ]:
tst_dl = learn.dls.test_dl(tst_df)

In [ ]:
preds, _ = learn.get_preds(dl=tst_dl)

In [ ]:
tst_df["Survived"] = (preds[:, 1] > 0.5).int()

In [ ]:
sub_df = tst_df[["PassengerId", "Survived"]]
sub_df.to_csv("submission.csv", index=False)

In [ ]:
sub = pd.read_csv("submission.csv")
sub

# Ensembling

In [ ]:
def ensemble():
    learn = tabular_learner(dls, metrics=accuracy, layers=[10, 10])
    with learn.no_bar(), learn.no_logging():
        learn.fit(16, lr=0.03)
    return learn.get_preds(dl=tst_dl)[0]

In [ ]:
learns = [ensemble() for _ in range(5)]

In [ ]:
ens_preds = torch.stack(learns).mean(0)

In [ ]:
tst_df["Survived"] = (ens_preds[:, 1] > 0.5).int()
sub_df = tst_df[["PassengerId", "Survived"]]
sub_df.to_csv("submission.csv", index=False)